In [31]:
import json
import random
import os
import numpy as np

def get_horizontal_frames(num_image, frames):
    equator_threshold = -0.4
    center_pt = np.zeros(3)
    
    # initialization of dict_list
    selected_frame_dict_list = []
    for i in range(num_image):
        s_f_dict = {}
        theta = 2*np.pi/num_image * i
        base_vector = np.array([np.sin(theta), np.cos(theta)])
        s_f_dict["base_vector"] = base_vector
        s_f_dict["selected_frame_inner_product"] = 0
        selected_frame_dict_list.append(s_f_dict)

    # select num_image frames
    for frame in frames:
        transform_matrix = frame["transform_matrix"]
        t_m_array = np.array(transform_matrix) # =~ c2w ?

        # z = t_m_array[2, 3] height??
        if t_m_array[2,3] <= equator_threshold: # TODO > <= ??
            continue

        # xy plane vector
        vector = t_m_array[[0, 1], 3] - center_pt[[0, 1]]
        # inner product =~ 1 -> select as a frame
        for i, s_f_dict in enumerate(selected_frame_dict_list):
            inner_product = np.dot(vector, s_f_dict["base_vector"])
            if s_f_dict["selected_frame_inner_product"] < inner_product:
                selected_frame_dict_list[i]["selected_frame"] = frame
                selected_frame_dict_list[i]["selected_frame_inner_product"] = inner_product

    selected_frames = [s_f_dict["selected_frame"] for s_f_dict in selected_frame_dict_list]
    return selected_frames

def modify_dict_data_horizontal(num_image, dict_data):
    frames = dict_data["frames"]

    selected_frames = get_horizontal_frames(num_image, frames)

    dict_data["frames"] = selected_frames
    return dict_data
    
def get_dict_data_from_json_path(input_json_path):
    with open(input_json_path, 'r') as f:
        dict_data = json.load(f)
    return dict_data

def save_dict_data(dict_data, output_json_path):
    with open(output_json_path, "w") as f:
        json.dump(dict_data, f)
        #json.dump(dict_data, f, indent=4)
        
def do_io_json_horizontal(input_dir_path, output_dir_path):
    input_json_path = input_dir_path + "train_all.json"
    dict_data = get_dict_data_from_json_path(input_json_path)
    
    for num_image in [3, 10, 25]:
        modified_dict_data = modify_dict_data_horizontal(num_image, dict_data)
        output_json_path = f"{output_dir_path}train_all_{num_image}_horizontal.json"
        save_dict_data(modified_dict_data, output_json_path)
    
def make_json_all_scene():
    path = "/home/ccl/Datasets/NeRF/ScanNerf/"
    scene_list = sorted(os.listdir(path))

    for scene in scene_list:
        input_dir_path = f"{path}{scene}/"

        # for test
        new_dir = f"./make_json_horizontal/train_horizontal"
        if not os.path.exists(new_dir):
            os.mkdir(new_dir)

        new_dir += f"/{scene}"
        if not os.path.exists(new_dir):
            os.mkdir(new_dir)
        output_dir_path = new_dir + "/"

        # for actual purpose
        # new_dir = f"{path}{scene}/" 
        
        do_io_json_horizontal(input_dir_path, output_dir_path)
        


In [11]:
make_json_all_scene()

In [12]:
# train_data_path = "/home/ccl/Datasets/NeRF/ScanNerf/"
# scene = "airplane1"
# input_dir = f"{train_data_path}{scene}/"
# new_dir = f"./make_json_horizontal/"
# open_and_modify_json_to_horizontal(input_dir, new_dir)

In [14]:
import json
from PIL import Image
import os


ckpt_path = "/home/ccl/Datasets/NeRF/ScanNerf/"
scene_list = sorted(os.listdir(ckpt_path))

def save_image_json(num_train_image):
    for scene in scene_list:
        json_dir_path = f"./make_json_horizontal/train_horizontal/{scene}/"

        with open(json_dir_path + f"train_all_{num_train_image}_horizontal.json", 'r') as f:
            dict_data = json.load(f)
        frames = dict_data["frames"]

        for frame in frames:
            file_path = frame["file_path"]
            img_path = f"{ckpt_path}{scene}/{file_path}.png"
            img = Image.open(img_path)

            output_dir = f"{json_dir_path}{num_train_image}"
            if not os.path.exists(output_dir):
                os.mkdir(output_dir)
            
            img.save(f"{output_dir}/{os.path.basename(file_path)}.png")
            
save_image_json(3)
for i in [3, 10, 25]:
    save_image_json(i)

In [22]:
def get_plot_camera_locations(dict_data):
    frames = dict_data["frames"]

    for frame in frames:
        file_path = frame["file_path"]
        img_path = f"{ckpt_path}{scene}/{file_path}.png"
        img = Image.open(img_path)

        output_dir = f"{json_dir_path}{num_train_image}"
        if not os.path.exists(output_dir):
            os.mkdir(output_dir)

        img.save(f"{output_dir}/{os.path.basename(file_path)}.png")
            
def

In [33]:
#
ckpt_path = "/home/ccl/Datasets/NeRF/ScanNerf/"
ckpt_path = "./make_json_horizontal/train_horizontal/"
scene = "airplane1"
num_image = 25
input_json_path = f"{ckpt_path}{scene}/train_all_{num_image}_horizontal.json"
dict_data = get_dict_data_from_json_path(input_json_path)

In [37]:
def show_plot_camera_locations(dict_data):
    frames = dict_data["frames"]
    
    t_m_arrays = [np.array(frame["transform_matrix"]) for frame in frames] # =~ c2ws?
    t_m_arrays
        
show_plot_camera_locations(dict_data)